# 1.   Проект для «Викишоп»

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
<br><br>Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
<br><br>
Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.</div>

## 1.1.   Подготовка

### 1.1.1.   Импорты библиотек

In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

import spacy
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1.1.2.   Изучение данных

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.shape

(159571, 2)

In [4]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

### 1.1.3.   Подготовка данных

#### 1.1.3.1.   Срез данных

#### 1.1.3.2.   Очистка текста

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Создадим функции очистки текста – регулярные выражения и лемматизация</div>

In [6]:
def clear_text(row):
    return ' '.join(re.sub(r'[^a-zA-Z ]', ' ', row).split())

In [7]:
def lemmatize_text(row):
    lm = WordNetLemmatizer()
    return ' '.join([lm.lemmatize(i) for i in nltk.word_tokenize(re.sub(r'[^a-zA-Z ]', ' ', row.lower()))])

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Закачаем английский вариант лемматизации</div>

In [8]:
nlp = spacy.load('en_core_web_sm')

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
И применим все к тексту</div>

In [9]:
data['clear_text'] = data['text'].apply(clear_text).apply(lemmatize_text)

In [10]:
data.head()

,text,toxic,clear_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Проверка на адекватность – нет ли строк с бесконечной пустотой</div>

In [11]:
data[data.clear_text == '']

,text,toxic,clear_text
4482,1993\n\n1994\n\n1995\n\n1996\n\n1997\n\n1998\n...,0,
6300,193.61.111.53 15:00,0,
17311,~ \n\n68.193.147.157,0,
52442,"14:53,",0,
53787,92.24.199.233|92.24.199.233]],0,
61758,"""\n\n 199.209.144.211 """,0,
82681,"""\n '''''' 2010/2013 """,0,


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Осталось некоторое количество пустых строк. Они нам могут помешать, так что стоит их убрать</div>

In [12]:
data = data[data.clear_text != '']

In [13]:
data[data.clear_text == '']

,text,toxic,clear_text


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Без пустых выглядит культурнее</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Текст готов, можно создавать признаки</div>

### 1.1.4.   Создание признаков

In [14]:
my_corpus = data.clear_text.values

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Разделяем данные на тренировочную и валидационную выборки</div>

#### 1.1.4.1.   Текстовый вариант признаков

In [15]:
features_train, features_valid, target_train, target_valid = train_test_split(data['clear_text'], data['toxic'], 
                                                                              random_state=12345, 
                                                                              # stratify=data['toxic'],
                                                                              test_size=0.2)

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Закачиваем стоп-слова</div>

In [16]:
stop_words = set(stopwords.words('english')) 

#### 1.1.4.2.   TF-IDF представление признаков

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
И создаем признаки на основе TF-IDF</div>

In [17]:
count_tfidf = TfidfVectorizer(ngram_range=(1, 3), 
                              stop_words=stop_words)

In [18]:
count_tfidf.fit(features_train)

TfidfVectorizer(ngram_range=(1, 3),
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [19]:
tfidf_train = count_tfidf.transform(features_train)

In [20]:
tfidf_valid = count_tfidf.transform(features_valid)

#### 1.1.4.3.   Векторное представление признаков

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Также, создаем признаки в виде вектора, для XGB</div>

In [21]:
def return_nlp_vector(row):
    return nlp(row).vector

In [22]:
vector_train = []
vector_valid = []

for row in tqdm(features_train):
    vector_train.append(nlp(row).vector)
    
for row in tqdm(features_valid):
    vector_valid.append(nlp(row).vector)

100%|██████████| 31913/31913 [07:43<00:00, 68.84it/s]


## 1.2.   Обучение

### 1.2.1.   Decision Tree

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Для начала обучим стандартное дерево решений</div>

In [23]:
%%time
pipe = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('model', DecisionTreeClassifier(random_state=12345))])
dtc_model = pipe.fit(features_train, target_train)

predicted = dtc_model.predict(features_valid)
f1 = f1_score(predicted, target_valid)
print(f1)

0.6824013939489942
CPU times: user 5min 39s, sys: 957 ms, total: 5min 40s
Wall time: 5min 40s


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Показатель уже хороший, но буквально чуть чуть не дотягивает до границы. Имеет смысл попробовать другую модель</div>

### 1.2.2.   Random Forest

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Возьмем RandoomForest и также запустим его со стандартными настройками – по опыту взаимодействия, показатели max_depth должны быть очень высокими, скорее всего даже четырехзначными, в то время как n_estimators при подборе гиперпараметров выдавал лучшие метрики на 3 деревьях</div>

In [24]:
%%time
pipe = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('model', RandomForestClassifier(random_state=12345))])
rfc_model = pipe.fit(features_train, target_train)

predicted = rfc_model.predict(features_valid)
f1 = f1_score(predicted, target_valid)
print(f1)

0.5822079314040729
CPU times: user 9min 24s, sys: 1.17 s, total: 9min 25s
Wall time: 9min 25s


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Показатель ниже, чем у Decision Tree</div>

### 1.2.3.   Logistic Regression

In [25]:
%%time
pipe = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('model', LogisticRegression())])
lr_model = pipe.fit(features_train, target_train)

predicted = lr_model.predict(features_valid)
f1 = f1_score(predicted, target_valid)
print(f1)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7521559805024373
CPU times: user 33.8 s, sys: 28 s, total: 1min 1s
Wall time: 1min 1s


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Нужный показатель метрики получен, но стоит проверить градиент</div>

### 1.2.4.   XGBoost

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
И, также, применим чистый XGB</div>

In [26]:
%%time
xgb_model = XGBClassifier()
xgb_model.fit(pd.DataFrame(vector_train), target_train)

predicted = xgb_model.predict(pd.DataFrame(vector_valid))
f1 = f1_score(predicted, target_valid)
print(f1)

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:17:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.37558896118465335
CPU times: user 8min 15s, sys: 718 ms, total: 8min 16s
Wall time: 8min 17s


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Модель градиентного бустинга с данной задачей не справилась, ответы менее осмыслены, чем рандомные</div>

## 1.3.   Выводы

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Были исследованы классические модели машинного обучения с задачей классификации – DecisionTreeClassifier, RandomForestClassifier и LogisticRegression</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Также были взяты модели с градиентным бустингом – CatBoostClassifier и XGBClassifier</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Данное исследование показало, что лучше всех с данной задачей справляются пайплайны. Они и быстрее, и метрики качества у них намного более высокие</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Из всех проверенных моделей лучше всего справляется логистическая регрессия, рекомендуется к применению</div>